# Parkinson’s Disease Detection from Speech Data

### Annika van Rooijen
### s0592559

In [2]:
# Libaries
import pandas as pd

# 1. Problemstellung

### Was ist Parkinson?
Der Morbus Parkinson ist eine chronisch fortschreitende, neurodegenerative Erkrankung, die unter anderem zu steifen Muskeln, verlangsamten Bewegungen und unkontrollierbarem Zittern führt [1]. 

### Was sind Ursachen für die Erkrankung?
Das primäre Parkinson-Syndrom geht von einer bestimmten Hirnregion aus, der sogenannten schwarzen Substanz (Substantia nigra) im Mittelhirn. Hier befinden sich spezielle Nervenzellen, die den Nervenbotenstoff Dopamin produzieren und mit ihm mit anderen Nervenzellen kommunizieren. Dopamin ist unter anderem wichtig für die Bewegungssteuerung. Durch die Parkinson-Erkrankung sterben immer mehr dieser Nervenzellen ab, es entwickeln sich ein Dopaminmangel und ein Ungleichgewicht der Nervenbotenstoffe im Gehirn [1]. 

Die Ursache für den Zelltod bei der Parkinson-Krankheit ist noch nicht eindeutig nachgewiesen. der primäre Parkinson macht etwa 75 Prozent aller Parkinson-Syndrome aus. Von diesem „klassischen“ Parkinson unterscheidet man die sehr seltenen genetischen Formen von Parkinson, das „Sekundäre Parkinson-Syndrom“, das z.B. durch Medikamente, Vergiftungen oder bestimmte Erkrankungen ausgelöst werden kann, und das „Atypische Parkinson-Syndrom“ als Folge verschiedenartiger anderer neurodegenerativer Erkrankungen [1]. 

### Was sind die Symptome der Parkinson Erkrankung? 
Zu den typischen Symptomen gehören das Zittern, weitere Bewegungsstörungen wie Steifheit der Muskeln, verlangsamte Bewegungen und Gleichgewichtsstörungen. Zusätzliche Symptome können das „Einfrieren“ von Bewegungen, Schwierigkeiten beim Sprechen und Schlucken, Störungen der vegetativen Funktionen (z. B. Blutdruck und Verdauung), Schlafstörungen, Depressionen und geistige Beeinträchtigungen bis hin zur Demenz sein [1].

Sprach- und Stimmveränderungen treten häufig auf und können bereits Jahre vor den klassischen motorischen Symptomen beobachtet werden. Diese frühen Veränderungen bieten ein Potenzial für eine frühere Erkennung und Überwachung der Krankheit, was wichtig wäre, da Diagnosen derzeit meist erst erfolgen, nachdem erheblicher neuronaler Verlust stattgefunden hat. Dies sind für Ärzt:innen schwer nur durchs hören erkennbar [2]. 

Zu den Veränderungen zählen eine verminderte Lautstärke (Hypophonie), eine monotone Sprechweise mit reduzierter Tonhöhen- und Lautstärkenvariation, eine verlangsamte oder unregelmäßige Sprechgeschwindigkeit sowie eine unpräzise Artikulation, bei der insbesondere Konsonanten weniger deutlich gebildet werden. Zusätzlich zeigen sich veränderte Pausenmuster, eine eingeschränkte Prosodie ((Prosodie = Betonung, Rhythmus, Melodie der Sprache)) mit flacher Betonung und reduziertem emotionalem Ausdruck sowie eine verminderte Stimmstabilität mit Schwankungen in Tonhöhe und Stimmqualität. Bei komplexeren Sprachaufgaben können außerdem Wortfindungsprobleme und eine reduzierte sprachliche Vielfalt auftreten. Diese Veränderungen sind oft so subtil, dass sie klinisch allein durch Zuhören schwer erkennbar sind und erst durch objektive, akustische und digitale Analyseverfahren zuverlässig erfasst werden können [2].

### Zielstellung
Die manuelle Analyse von Sprachaufnahmen ist häufig zeitaufwendig und mit einer begrenzten Zuverlässigkeit verbunden. Vor diesem Hintergrund stellt sich die Frage, ob künstliche Intelligenz in der Lage ist, Sprach- und Frequenzdaten automatisiert auszuwerten, um Hinweise auf eine neurodegenerative Erkrankung wie Parkinson zu erkennen. Ziel ist die Entwicklung eines KI-Modells, das anhand von Sprachaufnahmen und akustischen Merkmalen eine Parkinson-Erkrankung frühzeitig identifizieren kann.

## 2. Datenbeschaffung

Es gibt zahlreiche öffentlich verfügbare Datensätze zur Analyse von Parkinson anhand von Sprach- und Stimmmerkmalen. Die folgende Tabelle gibt einen Überblick über relevante Datensätze:

| Datensatzname | Datentyp | Inhalt | Größe / Umfang | Quelle |
|---------------|----------|--------|----------------|--------|
| UCI Parkinsons Dataset | Tabellarische Features | Frequenzmerkmale (Jitter, Shimmer, HNR, F0…) aus Sprachaufnahmen | 31 Personen, 195 Aufnahmen | https://archive.ics.uci.edu/dataset/174/parkinsons |
| UCI Parkinson Speech with Multiple Types of Audio | Tabellarische Features | Sustained Vowel, Wörter, Zahlen, Sätze | 40 Personen, mehrere hundert Audios | https://archive.ics.uci.edu/dataset/301/parkinson+speech+dataset |
| Figshare Parkinson Voice Samples | Audio (.wav) | Sustained Vowel /a/, Parkinson vs. Healthy | 100+ Aufnahmen | https://figshare.com/articles/dataset/23849127 |
| SJTU Parkinson Speech Dataset | Audio (.wav) | Verschiedene Sprachproben (Original-Speech) | > 150 Dateien | https://github.com/SJTU-YONGFU-RESEARCH-GRP/Parkinson-Patient-Speech-Dataset |
| Italian Parkinson’s Voice & Speech | Audio | Italienische Sprecher, Wörter, Vokale, Sätze | > 800 Aufnahmen, 65 Sprecher | https://ieee-dataport.org/open-access/italian-parkinsons-voice-and-speech |
| Mendeley Parkinson Acoustic Features | Tabellarische Features | Akustische Merkmale (MFCC, Jitter, Shimmer…) | ca. 200+ Einträge | https://data.mendeley.com/datasets/fjd6fcfkwn |
| NeuroVoz Parkinsonian Speech Corpus | Audio | Monologe, Vokale, Wörter, Wiederholungen, längere Sprache | 108 Sprecher |https://zenodo.org/records/10777657 (nicht öffentlich zugänglich)|
| PC-GITA Corpus | Audio | Spanische Sprecher, mehrere Sprachaufgaben, Parkinson vs. Healthy | 235 Sprecher | https://perception.csl.illinois.edu/PC-GITA.html (nicht öffentlich verfügbar)|
| MDVR-KCL Parkinson Voice Dataset | Audio | Sustained vowel phonation „a“, klinisch diagnostiziert | 40 Sprecher, 400+ Samples | https://www.kaggle.com/datasets/nutansingh/mdvr-kcl-dataset |
| mPower Parkinson Dataset | Audio + Sensorik | Smartphone-Sprachaufnahmen, longitudinal, Real-World-Daten | Tausende Teilnehmende | https://www.synapse.org/#!Synapse:syn4993293 |
| Parkinson Telemonitoring Dataset | Zeitreihen / Features | Sprachbasierte UPDRS-Messungen | 42 Personen, 5.875 Messungen | https://archive.ics.uci.edu/dataset/189/parkinsons+telemonitoring |
| German Parkinson Speech Corpus (nicht offen) | Audio | Deutsche Sprache, klinische Studien | Variabel | Nur über Forschungskooperation |
| RBD / Prodromal PD Speech Datasets | Audio | Sprachdaten von Hochrisikogruppen (prodromales Parkinson) | Forschungsdaten | Nicht öffentlich |



Für dieses Projekt wurden bewusst zwei unterschiedliche, sich ergänzende Datensätze ausgewählt. Der **UCI Parkinsons Datensatz** [5] dient als Einstieg und Benchmark, da er bereits extrahierte akustische Frequenzmerkmale enthält und eine schnelle, methodisch saubere Evaluation klassischer Machine-Learning-Modelle ermöglicht. Aufbauend auf diesen Ergebnissen wird der **Figshare-Datensatz mit Roh-Audioaufnahmen** [4] verwendet, um realistische Sprachsignale zu analysieren und moderne, audio-basierte Ansätze umzusetzen. Die Analyse der Audiodaten erfolgt mithilfe von **Parselmouth** [3], einer Python-Schnittstelle zu Praat, wodurch eine präzise und reproduzierbare Extraktion phonetischer und akustischer Merkmale aus den Sprachaufnahmen möglich ist. Dieser zweistufige Ansatz erlaubt einen strukturierten Übergang von tabellarischen Features zu echten Sprachsignalen und erhöht die methodische Nachvollziehbarkeit des Projekts.


# 3. Globale Eigenschaften: Oxford Parkinson's Disease Detection Dataset

Der Datensatz besteht aus 2 Tabellen:
1. parkinsons.data
2. parkinsons_updr.data

Der Datensatz ist schon sehr alt und stammt aus dem Jahr 2008.

## 3.1. parkinsons.data

Jede Zeile repräsentiert eine einzelne Sprachaufnahme, jede Spalte ein akustisches bzw. nichtlineares Stimmmerkmal, das aus der Aufnahme extrahiert wurde.

Die Tabelle besteht aus 195 Zeilen und 24 Spalten.
Der Datensatz hat keine fehlenden Werte. 

### Variablenübersicht

**MDVP (Multidimensional Voice Program)**
ist eine computergestützte Software zur objektiven akustischen Analyse der Stimmqualität

| Variable | Beschreibung | Datentyp |
|--------|-------------|----------|
| `name` | Eindeutiger Bezeichner der Person und der jeweiligen Aufnahme (z. B. `phon_R01_S01_1`) | String-Object |
| `MDVP:Fo(Hz)` | Durchschnittliche Grundfrequenz der Stimme (Fundamental Frequency) | Float |
| `MDVP:Fhi(Hz)` | Maximale Grundfrequenz der Stimme | Float |
| `MDVP:Flo(Hz)` | Minimale Grundfrequenz der Stimme | Float |
| `MDVP:Jitter(%)` | Relative Variation der Grundfrequenz | Float |
| `MDVP:Jitter(Abs)` | Absolute Variation der Grundfrequenz | Float |
| `MDVP:RAP` | Relative Average Perturbation (Kurzzeit-Jitter-Maß) | Float |
| `MDVP:PPQ` | Five-point Period Perturbation Quotient | Float |
| `Jitter:DDP` | Durchschnittliche absolute Differenz der Periodenlängen | Float |
| `MDVP:Shimmer` | Relative Amplitudenvariation | Float |
| `MDVP:Shimmer(dB)` | Amplitudenvariation in Dezibel | Float |
| `Shimmer:APQ3` | Drei-Punkte Amplituden-Perturbationsquotient | Float |
| `Shimmer:APQ5` | Fünf-Punkte Amplituden-Perturbationsquotient | Float |
| `MDVP:APQ` | Erweiterter Amplituden-Perturbationsquotient | Float |
| `Shimmer:DDA` | Durchschnittliche absolute Differenz der Amplituden | Float |
| `NHR` | Noise-to-Harmonics Ratio (Rauschanteil der Stimme) | Float |
| `HNR` | Harmonics-to-Noise Ratio (Klangreinheit der Stimme) | Float |
| `status` | Gesundheitsstatus: `1 = Parkinson`, `0 = gesund` | Integer (binär) |
| `RPDE` | Recurrence Period Density Entropy (nichtlineares Dynamikmaß) | Float |
| `DFA` | Detrended Fluctuation Analysis – fraktaler Skalierungsexponent | Float |
| `spread1` | Nichtlineares Maß der Grundfrequenzvariation | Float |
| `spread2` | Ergänzendes Maß zur Grundfrequenzvariation | Float |
| `D2` | Korrelationsdimension (nichtlineare Dynamik) | Float |
| `PPE` | Pitch Period Entropy – Unregelmäßigkeit der Grundfrequenz | Float |

---

### Zielvariable

- **`status`**  
  - `0` → gesunde Kontrollperson  
  - `1` → Parkinson-Erkrankung  

Ziel des Datensatzes ist die binäre Klassifikation von Parkinson-Erkrankten und gesunden Personen anhand von biomedizinischen Voice-Features. 



In [3]:
df_1 = pd.read_csv('data/oxford_PD_dataset/parkinsons.data')

In [4]:
# 195 rows, 24 colums
df_1.shape

(195, 24)

In [5]:
# keine NaN Values
df_1.isna().sum()

name                0
MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

In [6]:
print(df_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [7]:
df_1.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


### Feature Extraction
Zur Verbesserung der Analyse wurde die Spalte `name` in `record_id`umbenannt. Zusätzlich wurden aus dem zusammengesetzten Aufnahme-Identifikator weitere relevante Informationen extrahiert und in separate Spalten überführt.

In [8]:
# name-Spalte umbenennem
df_1 = df_1.rename(columns={'name': 'record_id'})

In [9]:
# subject number aus name-Spalte extrahieren

df_1['subject_number'] = (
    df_1['record_id']
    .str.extract(r'_S(\d+)_')
    .astype(int)
)


In [10]:
# Nummer der Aufnahme extrahieren 
df_1['record_number'] = (
    df_1['record_id']
    .str.extract(r'_(\d+)$')
    .astype(int)
)

In [11]:
# 'phon_' wird entfernt für bessere Lesbarkeit
df_1['record_id'] = df_1['record_id'].str.replace('phon_', '', regex=False)


In [12]:
id_cols = ['record_id', 'subject_number', 'record_number']
target_col = ['status']
feature_cols = [c for c in df_1.columns if c not in id_cols + target_col]

df_1= df_1[id_cols + feature_cols + target_col]

In [13]:
df_1.head()

,record_id,subject_number,record_number,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,...,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE,status
0,R01_S01_1,1,1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,...,0.06545,0.02211,21.033,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654,1
1,R01_S01_2,1,2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,...,0.09403,0.01929,19.085,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674,1
2,R01_S01_3,1,3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,...,0.08270,0.01309,20.651,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634,1
3,R01_S01_4,1,4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,...,0.08771,0.01353,20.644,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975,1
4,R01_S01_5,1,5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,...,0.10470,0.01767,19.649,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335,1


# 4. Exploratory Data Analysis


# 4. Featrure Engineering

# 5. Modelltraining

# 6. Modellevaluierung

# 7. Bonus

# 8. Fazit


# 9. Quellen
**[1]** Deutsche Gesellschaft für Parkinson und Bewegungsstörungen e. V. (o. J.). *Die Parkinson-Krankheit*. Abgerufen am *29. Dezember 2025*, von https://parkinson-gesellschaft.de/fuer-betroffene/die-parkinson-krankheit/

**[2]** Cao, F., Vogel, A. P., Gharahkhani, P., & Renteria, M. E. (2025). *Speech and language biomarkers for Parkinson’s disease prediction, early diagnosis and progression.* *npj Parkinson’s Disease, 11*(57). https://doi.org/10.1038/s41531-025-00913-4

**[3]** Jadoul, Y., Thompson, B., & de Boer, B. (2018). *Introducing Parselmouth: A Python interface to Praat*. *Journal of Phonetics, 71*, 1–15. [https://doi.org/10.1016/j.wocn.2018.07.001](https://doi.org/10.1016/j.wocn.2018.07.001) 

**[4]** Sakar, C. O., Serbes, G., & Gunduz, A. (2023). *Voice samples for patients with Parkinson’s disease and healthy controls* [Data set]. Figshare. [https://doi.org/10.6084/m9.figshare.23849127](https://doi.org/10.6084/m9.figshare.23849127)

**[5]** A Tsanas, MA Little, PE McSharry, LO Ramig (2009)
'Accurate telemonitoring of Parkinson.s disease progression by non-invasive speech tests',IEEE Transactions on Biomedical Engineering [https://doi.org/10.24432/C56C7T](https://doi.org/10.24432/C56C7T)

